Implement Newton's method for minimizing the function:

$f(x) = \frac{1}{2} x^T Q x - \sum_{i = 1}^n \log(x_i)$

subject to $Ax = b$.

Here $x \in R^n$ is the variable, $Q \in S^n$ is a positive definite matrix and $A \in R^{m \times n}$. To generate the problem data $Q$, let $Q = B^T B$ for some random $B \in R^{n \times n}$. To generate $A$ and $b$, let $x_0 = \boldsymbol{1}$, generate $A$ randomly and then set $b = Ax_0$.

In [1]:
#pip install numpy


In [1]:
import numpy as np

In [2]:
#Define the objective function

def f(x, Q):
    return 0.5 * x.T @ Q @ x - np.sum(np.log(x))


In [3]:
#Define the gradient of the objective function

def gradient_f(x, Q):
    return Q @ x - 1/x


In [4]:
#Define the Hessian of the objective function

def hessian_f(x, Q):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        H[i, i] = 1 / x[i]**2
    return Q + H


In [14]:
#Define Newton's method for optimization

def newtons_method(x_init, Q, A, b, tol = 1e-10, max_iter = 100):
    
    x = x_init
    alpha = 0.2
    beta = 0.5
    
    for _ in range(max_iter):
        
        grad = gradient_f(x, Q)
        hess = hessian_f(x, Q)
        
        #calculate coefficient matrix of KKT equation system
        m, n = A.shape
        top_row = np.hstack((hess, A.T))
        zeros_matrix = np.zeros((m, m)) 
        bottom_row = np.hstack((A, zeros_matrix))
        coeff = np.vstack((top_row, bottom_row))
        
        #calculate right-hand-side of KKT equation system
        zeros_vector = np.zeros((m, 1))
        rhs = np.vstack((-grad, zeros_vector))
        
        #solve the equation system for the Newton step
        sol = np.linalg.solve(coeff, rhs)
        newton_step = sol[0:len(grad)] #the Newton step is the first element of sol
        
        newton_step_norm = np.linalg.norm(newton_step)
        stop_criterion = (newton_step_norm ** 2)/2 #calculate stop criterion, a.k.a lambda squared divided by two
        print("Decrement: ", stop_criterion)
        print("Residual: " ,np.linalg.norm(A@x - b))
        
        if stop_criterion <= tol: #exit newtons method if the stop criterion is smaller than our tolerance level 
            break
        
        #if we the norm of the newton step isn't sufficiently small we iterate with step length t
        #making sure no component exit the dominion of x by adjusting step length t
        t = 1.0 #initial step length
        while True:
            x_new = x + t*newton_step #calculate new x
            if np.all(x_new > 0): #if all components are larger than zero we perform backtracking line search with alpha and beta
                while f(x_new, Q) > f(x, Q) + alpha*t*grad.T @ newton_step: #perform back tracking line search for as long as the new x does not give us a better function value
                    t *= beta 
                    x_new = x + t*newton_step 
                x = x_new 
                break #the outer while loop is broken as we have found an x that is in the dominion and minimizes the objective function
            else: #if the new x has components that are not larger than zero we decrease t
                t /= 2
              
    return x
        

In [15]:
#Define the conditions for your problem

#size of problem
n = 10
m = 5
#initial point
x0 = np.ones((n, 1))

#generate a symmetric positive definite matrix Q
B = np.random.rand(n, n)
Q = B.T @ B

#generate a random matrix A
A = np.random.rand(m, n)

#decide b
b = A @ x0


In [16]:
#Solve the problem using Newton's method

x_optimal = newtons_method(x0, Q, A, b)

print("The vector x that minimizes the objective function is \n", x_optimal)
print("The function value of that vector x is equal to \n", f(x_optimal, Q))



Decrement:  16.250509646204137
Residual:  0.0
Decrement:  5.258368391198106
Residual:  8.881784197001252e-16
Decrement:  0.6243018275673728
Residual:  1.3322676295501878e-15
Decrement:  0.038614504096929535
Residual:  2.5121479338940403e-15
Decrement:  0.00010389184850943375
Residual:  4.463041323674983e-15
Decrement:  1.1109427290340047e-08
Residual:  3.9968028886505635e-15
Decrement:  1.126308301932195e-15
Residual:  5.273267194922012e-15
The vector x that minimizes the objective function is 
 [[0.12993267]
 [1.26333393]
 [0.27451132]
 [3.20270049]
 [0.10538557]
 [0.84610506]
 [0.66394907]
 [0.95279649]
 [0.26852794]
 [1.56646077]]
The function value of that vector x is equal to 
 [[111.67517871]]
